# Libraries


In [77]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from tensorflow.keras import optimizers
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix


In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [78]:
from keras import regularizers
import os
mylist= os.listdir('C:\\Users\\kusha\\Biometrics\\Audio_Speech_Actors_01-24\\Actor_01')
type(mylist)

list

In [79]:
print(mylist[1])

03-01-01-01-01-02-01.wav


In [80]:
feeling_list=[]
for item in mylist:
    if item[6:-16]=='02' and int(item[18:-4])%2==0:
        feeling_list.append('female_calm')
    elif item[6:-16]=='02' and int(item[18:-4])%2==1:
        feeling_list.append('male_calm')
    elif item[6:-16]=='03' and int(item[18:-4])%2==0:
        feeling_list.append('female_happy')
    elif item[6:-16]=='03' and int(item[18:-4])%2==1:
        feeling_list.append('male_happy')
    elif item[6:-16]=='04' and int(item[18:-4])%2==0:
        feeling_list.append('female_sad')
    elif item[6:-16]=='04' and int(item[18:-4])%2==1:
        feeling_list.append('male_sad')
    elif item[6:-16]=='05' and int(item[18:-4])%2==0:
        feeling_list.append('female_angry')
    elif item[6:-16]=='05' and int(item[18:-4])%2==1:
        feeling_list.append('male_angry')
    elif item[6:-16]=='06' and int(item[18:-4])%2==0:
        feeling_list.append('female_fearful')
    elif item[6:-16]=='06' and int(item[18:-4])%2==1:
        feeling_list.append('male_fearful')
    elif item[:1]=='a':
        feeling_list.append('male_angry')
    elif item[:1]=='f':
        feeling_list.append('male_fearful')
    elif item[:1]=='h':
        feeling_list.append('male_happy')
    #elif item[:1]=='n':
        #feeling_list.append('neutral')
    elif item[:2]=='sa':
        feeling_list.append('male_sad')

In [81]:

import os
import pandas as pd
import glob 

In [82]:
labels = pd.DataFrame(feeling_list)
labels[:10]

,0
0,male_calm
1,male_calm
2,male_calm
3,male_calm
4,male_calm
5,male_calm
6,male_calm
7,male_calm
8,male_happy
9,male_happy


# Extracting audio features.... hopefully

In [83]:
df = pd.DataFrame(columns=['feature'])
bookmark=0
for index,y in enumerate(mylist):
    if mylist[index][6:-16]!='01' and mylist[index][6:-16]!='07' and mylist[index][6:-16]!='08' and mylist[index][:2]!='su' and mylist[index][:1]!='n' and mylist[index][:1]!='d':
        X, sample_rate = librosa.load('C:\\Users\\kusha\\Biometrics\\Audio_Speech_Actors_01-24\\Actor_01\\'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                            sr=sample_rate, 
                                            n_mfcc=13),
                        axis=0)
        feature = mfccs
        #[float(i) for i in feature]
        #feature1=feature[:135]
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1        
df[:5]

,feature
0,"[-70.26777, -70.26777, -70.26777, -70.26777, -..."
1,"[-67.55739, -67.55739, -67.55739, -67.55739, -..."
2,"[-69.67329, -69.693306, -69.693306, -69.693306..."
3,"[-69.0514, -69.0514, -69.0514, -69.0514, -69.0..."
4,"[-73.84137, -73.84137, -73.84137, -73.71965, -..."


In [84]:
df3 = pd.DataFrame(df['feature'].values.tolist())
newdf = pd.concat([df3,labels], axis=1)
rnewdf = newdf.rename(index=str, columns={"0": "label"})
rnewdf[:5]

,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,0
0,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,...,-57.447464,-58.896500,-58.750996,-57.405678,-60.078484,-63.426800,-62.638542,-61.082741,-60.234661,male_calm
1,-67.557388,-67.557388,-67.557388,-67.557388,-67.557388,-67.557388,-65.239815,-65.536194,-67.557388,-67.557388,...,-57.270752,-56.743652,-56.140930,-57.532017,-59.493046,-67.149849,-67.498711,-66.086418,-66.205048,male_calm
2,-69.673286,-69.693306,-69.693306,-69.693306,-69.693306,-69.693306,-69.693306,-69.620773,-69.693306,-68.906570,...,-61.018559,-60.341145,-63.465332,-64.500137,-61.646843,-58.001488,-58.848484,-62.603935,-61.121773,male_calm
3,-69.051399,-69.051399,-69.051399,-69.051399,-69.051399,-68.754860,-69.051399,-69.051399,-69.051399,-68.359085,...,-57.681641,-56.301266,-58.650131,-63.881088,-63.749409,-65.222473,-65.308250,-67.008553,-68.636299,male_calm
4,-73.841370,-73.841370,-73.841370,-73.719650,-73.841370,-73.841370,-73.841370,-73.303635,-72.806808,-73.841370,...,-50.568867,-53.505844,-54.222256,-51.545525,-52.573784,-54.786297,-56.344276,-57.508217,-56.577332,male_calm


In [85]:
from sklearn.utils import shuffle
rnewdf = shuffle(newdf)
rnewdf[:10]

,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,0
12,-58.731003,-58.746517,-58.048584,-57.370796,-58.199924,-59.706638,-59.876179,-59.998066,-59.936649,-60.248569,...,-50.477974,-50.955284,-55.110638,-53.636642,-57.134220,-58.249916,-56.417946,-54.576748,-51.933964,male_happy
4,-73.841370,-73.841370,-73.841370,-73.719650,-73.841370,-73.841370,-73.841370,-73.303635,-72.806808,-73.841370,...,-50.568867,-53.505844,-54.222256,-51.545525,-52.573784,-54.786297,-56.344276,-57.508217,-56.577332,male_calm
34,-59.449245,-60.975838,-62.336349,-62.336349,-62.336349,-61.444695,-62.336349,-62.336349,-62.336105,-62.336349,...,-61.327873,-61.667442,-59.324917,-60.374935,-61.389545,-62.336349,-62.319279,-59.312874,-60.265171,male_fearful
23,-61.560623,-61.560623,-61.560623,-61.560623,-61.560623,-61.560623,-61.560623,-61.560623,-61.560623,-61.560623,...,-53.844570,-54.260578,-55.809025,-54.627144,-53.186848,-53.998234,-53.535164,-57.203739,-59.989193,male_sad
33,-61.087368,-61.579323,-61.580788,-61.288128,-61.081142,-60.591736,-59.902363,-59.477730,-60.251350,-61.044029,...,-53.662144,-53.636868,-55.863472,-56.614315,-56.009319,-54.970852,-54.317688,-56.101189,-56.822609,male_fearful
6,-73.254967,-73.254967,-73.254967,-73.254967,-68.774429,-69.380402,-73.254967,-73.254967,-73.254967,-73.254967,...,-26.499784,-27.143425,-28.207592,-27.193054,-27.315336,-27.431173,-26.716452,-24.115793,-21.873396,male_calm
25,-55.623817,-55.623817,-55.570381,-53.938660,-52.901047,-54.380760,-54.381161,-55.478569,-55.612228,-55.888786,...,-31.432295,-33.775314,-36.061378,-38.142239,-38.785397,-39.644428,-41.592487,-45.736794,-42.881668,male_angry
32,-45.030743,-46.782719,-47.775520,-45.638206,-45.101456,-48.043095,-48.737019,-48.716675,-49.198879,-49.100117,...,-56.838863,-54.096439,-53.427292,-59.046059,-57.349792,-58.454384,-59.379395,-60.010826,-60.010826,male_fearful
0,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,...,-57.447464,-58.896500,-58.750996,-57.405678,-60.078484,-63.426800,-62.638542,-61.082741,-60.234661,male_calm
20,-65.755875,-65.755875,-65.755875,-65.755875,-65.755875,-65.755875,-65.755875,-65.755875,-65.755875,-65.755875,...,-41.763081,-43.194901,-44.881516,-45.717583,-48.605293,-53.018700,-49.683552,-51.019882,-57.109806,male_sad


In [86]:
rnewdf=rnewdf.fillna(0)

# testing and training

In [87]:
newdf1 = np.random.rand(len(rnewdf)) < 0.8
train = rnewdf[newdf1]
test = rnewdf[~newdf1]
train[20:30]

,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,0
2,-69.673286,-69.693306,-69.693306,-69.693306,-69.693306,-69.693306,-69.693306,-69.620773,-69.693306,-68.906570,...,-61.018559,-60.341145,-63.465332,-64.500137,-61.646843,-58.001488,-58.848484,-62.603935,-61.121773,male_calm
15,-57.414993,-57.414993,-57.414993,-57.414993,-57.414993,-57.414993,-57.414993,-57.414993,-57.414993,-57.414993,...,-27.892962,-29.425554,-31.255730,-34.011841,-34.322582,-33.564938,-36.074425,-37.658699,-38.918003,male_happy
19,-68.049080,-68.049080,-68.049080,-68.049080,-68.049080,-68.049080,-68.049080,-68.049080,-68.049080,-68.049080,...,-68.049080,-68.049080,-68.049080,-68.049080,-68.049080,-68.049080,-68.049080,-68.049080,-68.049080,male_sad
11,-65.243347,-65.243347,-65.243347,-65.243347,-65.243347,-65.243347,-65.243347,-65.243347,-65.243347,-65.117577,...,-61.924271,-56.156918,-55.677975,-61.117817,-58.568733,-57.136806,-58.619907,-60.068314,-57.350605,male_happy
8,-63.311062,-63.072487,-63.412422,-63.796757,-63.581989,-58.921223,-57.955044,-61.224972,-63.782932,-63.796757,...,-62.505020,-59.518867,-60.482346,-58.338570,-58.965298,-63.520664,-63.769352,-63.829861,-62.814236,male_happy
37,-43.197392,-44.238605,-43.996212,-44.132359,-44.267864,-44.346043,-44.142902,-44.255211,-43.666569,-43.591591,...,-33.476284,-33.367661,-32.961880,-31.957905,-31.085176,-31.186022,-30.768852,-19.886076,-12.408971,male_fearful
29,-42.103939,-42.229004,-44.166603,-43.172729,-43.590572,-42.570866,-43.202087,-43.710556,-43.444092,-43.015808,...,-22.873636,-20.374043,-17.853765,-16.383234,-14.637953,-18.462688,-21.157621,-10.558844,-3.886741,male_angry
3,-69.051399,-69.051399,-69.051399,-69.051399,-69.051399,-68.754860,-69.051399,-69.051399,-69.051399,-68.359085,...,-57.681641,-56.301266,-58.650131,-63.881088,-63.749409,-65.222473,-65.308250,-67.008553,-68.636299,male_calm
21,-62.104851,-62.104851,-62.104851,-62.037937,-62.104851,-62.104851,-62.104851,-62.104851,-62.104851,-62.104851,...,-60.357834,-61.090469,-62.026588,-61.804722,-61.251228,-59.135616,-61.459846,-62.104851,-62.104851,male_sad
26,-55.663086,-56.170334,-56.157845,-54.663475,-55.166775,-56.156944,-56.156944,-56.156944,-56.156944,-56.156944,...,-56.156944,-56.156944,-56.156944,-55.844219,-56.156944,-56.156944,-56.156944,-56.156944,-56.156944,male_angry


In [88]:
trainfeatures = train.iloc[:, :-1]
trainlabel = train.iloc[:, -1:]
testfeatures = test.iloc[:, :-1]
testlabel = test.iloc[:, -1:]

In [89]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

X_train = np.array(trainfeatures)
y_train = np.array(trainlabel)
X_test = np.array(testfeatures)
y_test = np.array(testlabel)

lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

C:\Users\kusha\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [90]:
y_train

array([[0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [91]:
X_train.shape

(32, 216)

#CNN 

In [92]:
x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)

In [97]:
model = Sequential()

model.add(Conv1D(256, 5,padding='same',
                 input_shape=(216,1)))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
#model.add(Conv1D(128, 5,padding='same',))
#model.add(Activation('relu'))
#model.add(Conv1D(128, 5,padding='same',))
#model.add(Activation('relu'))
#model.add(Dropout(0.2))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation('softmax'))
opt = keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)
model.summary()


AttributeError: module 'keras.optimizers' has no attribute 'RMSprop'

In [38]:
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

NameError: name 'opt' is not defined

In [57]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=16, epochs=700, validation_data=(x_testcnn, y_test))

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [ ]:
model_name = 'Emotion_Voice_Detection_Model.h5'
save_dir = os.path.join(os.getcwd(), 'saved_models')
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)